In [2]:
import pandas as pd
import numpy as np
import time

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from sklearn import linear_model, svm
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold,\
    StratifiedShuffleSplit, cross_val_score

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,\
    roc_auc_score, roc_curve, classification_report


import xgboost as xgb
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

#import eli5

import graphviz
from IPython.display import Image
import pydotplus

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

pd.set_option('display.max_columns',99)
pd.set_option('display.max_rows',300)

In [2]:
pip install pydotplus

Note: you may need to restart the kernel to use updated packages.


### Import and Processes Datasets

In [3]:
#READ CSVs

#15,20,25,30 whole and unique

train_unique_15_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/15/unique_train_cleaned_15.csv')
train_whole_15_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/15/whole_train_cleaned_15.csv')
test_unique_15_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/15/unique_test_cleaned_15.csv')

train_unique_20_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/20/unique_train_cleaned_20.csv')
train_whole_20_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/20/whole_train_cleaned_20.csv')
test_unique_20_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/20/unique_test_cleaned_20.csv')

train_unique_25_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/25/unique_train_cleaned_25.csv')
train_whole_25_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/25/whole_train_cleaned_25.csv')
test_unique_25_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/25/unique_test_cleaned_25.csv')

train_unique_30_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/30/unique_train_cleaned_30.csv')
train_whole_30_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/30/whole_train_cleaned_30.csv')
test_unique_30_orig = pd.read_csv('dataset_diabetes/experiment/new/standardized splits/30/unique_test_cleaned_30.csv')

In [4]:
#make copies. #Re-Run this cell to reset version and split selections.

train_unique_15 = train_unique_15_orig.copy()
test_unique_15 = test_unique_15_orig.copy()
train_whole_15 = train_whole_15_orig.copy()

train_unique_20 = train_unique_20_orig.copy()
test_unique_20 = test_unique_20_orig.copy()
train_whole_20 = train_whole_20_orig.copy()

train_unique_25 = train_unique_25_orig.copy()
test_unique_25 = test_unique_25_orig.copy()
train_whole_25 = train_whole_25_orig.copy()

train_unique_30 = train_unique_30_orig.copy()
test_unique_30 = test_unique_30_orig.copy()
train_whole_30 = train_whole_30_orig.copy()

In [5]:
def print_report(y_actual, y_pred, y_pred_proba):
    tn, fp, fn, tp = confusion_matrix(y_actual, y_pred).ravel()
    fpr = fp/(fp+tn)
    auc = roc_auc_score(y_actual, y_pred_proba)
    print('AUC:%.3f'%auc)
    print(f'False positive rate is {fpr}')
    print('tp',tp,'fp',fp,'fn',fn,'tn',tn)

In [6]:
#prepare dfs for model input. #Re-Run this cell to reset version and split selections.
def prep_dfs(split,train_unique,train_whole,test_unique):
    
    train_unique_y = train_unique['readmitted']
    unique_encounter_patient = train_unique[['encounter_id','patient_nbr']]
    train_unique.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    train_whole_y = train_whole['readmitted']
    whole_encounter_patient = train_whole[['encounter_id','patient_nbr']]
    train_whole.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    test_unique_y = test_unique['readmitted']
    test_unique.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    print('{split} percent X_train_unique shape is {shape}'.format(split = split, shape = train_unique.shape))
    print('{split} percent y_train_unique shape is {shape}'.format(split = split, shape = train_unique_y.shape))
    print('{split} percent X_train_whole shape is {shape}'.format(split = split, shape = train_whole.shape))
    print('{split} percent y_train_whole shape is {shape}'.format(split = split, shape = train_whole_y.shape))
    print('{split} percent X_test_unique shape is {shape}'.format(split = split, shape = test_unique.shape))
    print('{split} percent y_test_unique shape is {shape}'.format(split = split, shape = test_unique_y.shape))
    print(' ')
    
    #appy SMOTE
#     smt = SMOTE()
#     train_unique, train_unique_y = smt.fit_sample(train_unique, train_unique_y)
#     train_whole, train_whole_y = smt.fit_sample(train_whole, train_whole_y)

# #     nr = NearMiss()
# #     X_nearmissW, y_nearmissW = nr.fit_sample(X_trainW, y_trainW)
# #     X_nearmissU, y_nearmissU = nr.fit_sample(X_trainU, y_trainU)

#     print('The number of observations in WHOLE_train for each class using SMOTE is now {}'.format(np.bincount(train_whole_y)))
#     print('The number of observations in UNIQUE_train for each class using SMOTE is now {}'.format(np.bincount(train_unique_y)))
    #print('The number of observations in the WHOLE dataset for each class using Near Miss are now {}'.format(np.bincount(y_nearmissW)))
    #print('The number of observations in the UNIQUE dataset for each class using Near Miss are now {}'.format(np.bincount(y_nearmissU)))
    print(' ')
    
    return train_unique, train_unique_y, unique_encounter_patient, train_whole, train_whole_y,\
        whole_encounter_patient,test_unique,test_unique_y

In [7]:
#create dataframes for models. #Re-Run this cell to reset version and split selections.
X_train_unique_15, y_train_unique_15, unique_15_encounter_patient, X_train_whole_15,\
y_train_whole_15, whole_15_encounter_patient,X_test_unique_15,y_test_unique_15 = prep_dfs(15,train_unique_15,\
                                                                       train_whole_15,\
                                                                       test_unique_15)

X_train_unique_20, y_train_unique_20, unique_20_encounter_patient, X_train_whole_20,\
y_train_whole_20, whole_20_encounter_patient,X_test_unique_20,y_test_unique_20 = prep_dfs(20,train_unique_20,\
                                                                       train_whole_20,\
                                                                       test_unique_20)

X_train_unique_25, y_train_unique_25, unique_25_encounter_patient, X_train_whole_25,\
y_train_whole_25, whole_25_encounter_patient,X_test_unique_25,y_test_unique_25 = prep_dfs(25,train_unique_25,\
                                                                        train_whole_25,\
                                                                        test_unique_25)

X_train_unique_30, y_train_unique_30, unique_30_encounter_patient, X_train_whole_30,\
y_train_whole_30, whole_30_encounter_patient,X_test_unique_30,y_test_unique_30 = prep_dfs(30,train_unique_30,\
                                                                       train_whole_30,\
                                                                       test_unique_30)

15 percent X_train_unique shape is (61674, 91)
15 percent y_train_unique shape is (61674,)
15 percent X_train_whole shape is (84422, 91)
15 percent y_train_whole shape is (84422,)
15 percent X_test_unique shape is (13835, 91)
15 percent y_test_unique shape is (13835,)
 
 
20 percent X_train_unique shape is (58793, 91)
20 percent y_train_unique shape is (58793,)
20 percent X_train_whole shape is (79456, 91)
20 percent y_train_whole shape is (79456,)
20 percent X_test_unique shape is (18019, 91)
20 percent y_test_unique shape is (18019,)
 
 
25 percent X_train_unique shape is (55815, 91)
25 percent y_train_unique shape is (55815,)
25 percent X_train_whole shape is (74490, 91)
25 percent y_train_whole shape is (74490,)
25 percent X_test_unique shape is (22049, 91)
25 percent y_test_unique shape is (22049,)
 
 
30 percent X_train_unique shape is (52825, 91)
30 percent y_train_unique shape is (52825,)
30 percent X_train_whole shape is (69524, 91)
30 percent y_train_whole shape is (69524,)
3

In [8]:
#dictionary to store all the different dataframes with different splits.

#Re-Run this cell to reset version and split selections.

versions = {'X_train':{'15w':X_train_whole_15,'15u':X_train_unique_15,\
                       '20w':X_train_whole_20,'20u':X_train_unique_20,\
                       '25w':X_train_whole_25,'25u':X_train_unique_25,\
                       '30w':X_train_whole_30,'30u':X_train_unique_30},\
            'y_train':{'15w':y_train_whole_15,'15u':y_train_unique_15,\
                       '20w':y_train_whole_20,'20u':y_train_unique_20,\
                       '25w':y_train_whole_25,'25u':y_train_unique_25,\
                       '30w':y_train_whole_30,'30u':y_train_unique_30},
            'X_test':{ '15u':X_test_unique_15,\
                       '20u':X_test_unique_20,\
                       '25u':X_test_unique_25,\
                       '30u':X_test_unique_30},\
            'y_test':{ '15u':y_test_unique_15,\
                       '20u':y_test_unique_20,\
                       '25u':y_test_unique_25,\
                       '30u':y_test_unique_30}}

In [9]:
#declare which version to run in model below:

def choose_version(data_split,selection):

    X_train = versions['X_train'][str(data_split)+selection] 
    y_train = versions['y_train'][str(data_split)+selection]
    X_test = versions['X_test'][str(data_split)+'u']
    y_test = versions['y_test'][str(data_split)+'u']
    
    return X_train, y_train, X_test, y_test

In [10]:
def apply_smote(X_train,y_train):
    
    smt = SMOTE()
    X_train_SMOTE, y_train_SMOTE = smt.fit_sample(X_train, y_train)
    
    print('The number of observations for each class using SMOTE is now {}'.format(np.bincount(y_train_SMOTE)))
    return X_train_SMOTE, y_train_SMOTE

### ==========SELECT SPLIT and VERSION, and SMOTE==========

In [11]:
#++++++++++++++++RUN THIS CELL TO SET WHICH SPLIT VERSION:++++++++++++++++++++

X_train, y_train, X_test, y_test = choose_version(15,'w')

# X_train, y_train, X_test, y_test = choose_version(25,'u')
# X_train, y_train, X_test, y_test = choose_version(30,'w')

#++++++++++++++++RUN THIS CELL TO SET WHICH SPLIT VERSION:++++++++++++++++++++

In [14]:
####################################
###ONLY RUN IF APPLYING SMOTE!!#####
####################################
X_train_SMOTE, y_train_SMOTE = apply_smote(X_train,y_train)


The number of observations for each class using SMOTE is now [74836 74836]


### =====================================================

## MODELING 
**(Logistic Regression, Decision Tree, Random Forest, XGBoost, LightGBM)**

### Logistic Regression

In [ ]:
logreg = LogisticRegression(fit_intercept=True, penalty='l2')
logreg.fit(X_train, y_train)
y_test_predict = logreg.predict(X_test)
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
# Print Classification Report
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_logreg = accuracy_score(y_test, y_test_predict)
precision_logreg = precision_score(y_test, y_test_predict)
recall_logreg = recall_score(y_test, y_test_predict)
auc_logreg = roc_auc_score(y_test, y_test_predict)
TPR_logreg=TPR
FPR_logreg=FPR
TNR_logreg=TNR

In [ ]:
coef_base=pd.DataFrame(zip(X_train.columns, np.transpose(logreg.coef_)),columns=['Features','Coefficients']).sort_values(by='Coefficients',ascending=False)

In [ ]:
coef_base.head(10)

#### Apply SMOTE

In [ ]:
# Run the crossvalidation following by fit the SMOTE balanced data

logreg = LogisticRegression(solver = 'liblinear',fit_intercept=True, penalty='l1')
logreg.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
# Check with the validation testset
y_test_predict = logreg.predict(X_test)
probability = logreg.predict_proba(X_test)

In [ ]:
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_logreg = accuracy_score(y_test, y_test_predict)
precision_logreg = precision_score(y_test, y_test_predict)
recall_logreg = recall_score(y_test, y_test_predict)
auc_logreg = roc_auc_score(y_test, y_test_predict)
TPR_logreg=TPR
FPR_logreg=FPR
TNR_logreg=TNR

#### Apply 'Class_weight' = balanced

In [ ]:
logreg = LogisticRegression(solver = 'liblinear',fit_intercept=True, penalty='l1',class_weight='balanced')
logreg.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
# Check with the validation testset
y_test_predict = logreg.predict(X_test)
probability = logreg.predict_proba(X_test)

In [ ]:
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_logreg = accuracy_score(y_test, y_test_predict)
precision_logreg = precision_score(y_test, y_test_predict)
recall_logreg = recall_score(y_test, y_test_predict)
auc_logreg = roc_auc_score(y_test, y_test_predict)
TPR_logreg=TPR
FPR_logreg=FPR
TNR_logreg=TNR

In [ ]:
coef_SMOTE=pd.DataFrame(zip(X_train_SMOTE.columns, np.transpose(logreg.coef_)),columns=['Features','Coefficients']).sort_values(by='Coefficients',ascending=False)

In [ ]:
coef_SMOTE.head(10)

#### Apply RandomizedSearch

In [ ]:
# Apply randomizedsearch to find the optimum "C"

from scipy.stats import uniform

logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions, random_state=0)
search = clf.fit(X_train_SMOTE, y_train_SMOTE)
search.best_params_


In [ ]:
logreg = LogisticRegression(solver = 'liblinear',fit_intercept=True, penalty='l1',class_weight='balanced',C=2.195254015709299)
logreg.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
# Check with the validation testset
y_test_predict = logreg.predict(X_test)
probability = logreg.predict_proba(X_test)

In [ ]:
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_logreg = accuracy_score(y_test, y_test_predict)
precision_logreg = precision_score(y_test, y_test_predict)
recall_logreg = recall_score(y_test, y_test_predict)
auc_logreg = roc_auc_score(y_test, y_test_predict)
TPR_logreg=TPR
FPR_logreg=FPR
TNR_logreg=TNR

In [ ]:
coef_random_search=pd.DataFrame(zip(X_train_SMOTE.columns, np.transpose(logreg.coef_)),columns=['Features','Coefficients']).sort_values(by='Coefficients',ascending=False)

In [ ]:
coef_random_search.head(10)

### Decision Tree Clssifier - "Entropy"

In [ ]:
dt_en = DecisionTreeClassifier(max_depth=28, criterion = "entropy", min_samples_split=10)
dt_en.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
y_test_predict = dt_en.predict(X_test)

In [ ]:
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_dt_en = accuracy_score(y_test, y_test_predict)
precision_dt_en = precision_score(y_test, y_test_predict)
recall_dt_en = recall_score(y_test, y_test_predict)
auc_dt_en = roc_auc_score(y_test, y_test_predict)
TPR_dt_en=TPR
FPR_ldt_en=FPR
TNR_dt_en=TNR

In [ ]:
dot_dt_q2 = tree.export_graphviz(dt_en, out_file="dt_q2.dot", feature_names=X_train.columns, max_depth=2,
                                 class_names=["No","Readm"], filled=True, rounded=True, special_characters=True)
graph_dt_q2 = pydotplus.graph_from_dot_file('dt_q2.dot')
Image(graph_dt_q2.create_png())

In [ ]:
# Create list of top most features based on importance
feature_names = X_train.columns
feature_imports = dt_en.feature_importances_
most_imp_features = pd.DataFrame([f for f in zip(feature_names,feature_imports)], columns=["Feature", "Importance"]).nlargest(10, "Importance")
most_imp_features.sort_values(by="Importance", inplace=True)
print(most_imp_features)
plt.figure(figsize=(10,6))
plt.barh(range(len(most_imp_features)), most_imp_features.Importance, align='center', alpha=0.8)
plt.yticks(range(len(most_imp_features)), most_imp_features.Feature, fontsize=14)
plt.xlabel('Importance')
plt.title('Most important features - Decision Tree (entropy function)')
plt.show()

### Random Forest Classifier : "Gini"

In [ ]:
rf_gini = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "gini", min_samples_split=10)
rf_gini.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
y_test_predict = rf_gini.predict(X_test)

pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_rf_g = accuracy_score(y_test, y_test_predict)
precision_rf_g = precision_score(y_test, y_test_predict)
recall_rf_g = recall_score(y_test, y_test_predict)
auc_rf_g = roc_auc_score(y_test, y_test_predict)

TPR_rf_g=TPR
FPR_rf_g=FPR
TNR_rf_g=TNR

In [ ]:
# Create list of top most features based on importance
feature_names = X_train.columns
feature_imports = dt_en.feature_importances_
most_imp_features = pd.DataFrame([f for f in zip(feature_names,feature_imports)], columns=["Feature", "Importance"]).nlargest(10, "Importance")
most_imp_features.sort_values(by="Importance", inplace=True)
print(most_imp_features)
plt.figure(figsize=(10,6))
plt.barh(range(len(most_imp_features)), most_imp_features.Importance, align='center', alpha=0.8)
plt.yticks(range(len(most_imp_features)), most_imp_features.Feature, fontsize=14)
plt.xlabel('Importance')
plt.title('Most important features - Random Forest (Gini)')
plt.show()

### Random Forest Classifier : "Entropy"

In [ ]:
rf_en = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "entropy", min_samples_split=10)
rf_en.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
y_test_predict = rf_en.predict(X_test)
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_test_predict, name = 'Predict'), margins = True)

In [ ]:
report=classification_report(y_test, y_test_predict)
print(report)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_predict).ravel()
print(tn, fp, fn, tp)  # 1 1 1 1

TPR=tp/(tp+fn)
FPR=fp/(fp+tn)
TNR=tn/(fp+tn)

In [ ]:
print("Accuracy is {0:.3f}".format(accuracy_score(y_test, y_test_predict)))
print("Precision is {0:.3f}".format(precision_score(y_test, y_test_predict)))
print("Recall is {0:.3f}".format(recall_score(y_test, y_test_predict)))
print("AUC is {0:.3f}".format(roc_auc_score(y_test, y_test_predict)))
print("TPR is {0:.3f}".format(TPR))
print("FPR is {0:.3f}".format(FPR))
print("TNR is {0:.3f}".format(TNR))

In [ ]:
accuracy_rf_en = accuracy_score(y_test, y_test_predict)
precision_rf_en = precision_score(y_test, y_test_predict)
recall_rf_en = recall_score(y_test, y_test_predict)
auc_rf_en = roc_auc_score(y_test, y_test_predict)

TPR_rf_en=TPR
FPR_rf_en=FPR
TNR_rf_en=TNR

In [ ]:
# Create list of top most features based on importance
feature_names = X_train.columns
feature_imports = rf_en.feature_importances_
most_imp_features = pd.DataFrame([f for f in zip(feature_names,feature_imports)], columns=["Feature", "Importance"]).nlargest(10, "Importance")
most_imp_features.sort_values(by="Importance", inplace=True)
plt.figure(figsize=(10,6))
plt.barh(range(len(most_imp_features)), most_imp_features.Importance, align='center', alpha=0.8)
plt.yticks(range(len(most_imp_features)), most_imp_features.Feature, fontsize=14)
plt.xlabel('Importance')
plt.title('Most important features - Random Forest (entropy)')
plt.show()

### XGBoost

In [15]:

xgbc = xgb.XGBClassifier()

In [16]:
#fit X_train, y_train
xgbc.fit(X_train_SMOTE,y_train_SMOTE)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
thresh=0.5
y_train_pred_proba = xgbc.predict_proba(X_train_SMOTE)[:,1]
y_test_pred_proba = xgbc.predict_proba(X_test)[:,1]

y_train_pred = xgbc.predict(X_train_SMOTE)
y_test_pred = xgbc.predict(X_test)

print('XGBoost')
print('Training:')
print_report(y_train_SMOTE, y_train_pred, y_train_pred_proba)
# print('Validation:')
# print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('')
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)

In [ ]:
sorted(zip(X_train_SMOTE.columns,xgbc.feature_importances_), key= lambda t:t[1], reverse=True)

### **parameter tuning**

In [ ]:
xgbc1 = xgb.XGBClassifier()
xgbc1.set_params(learning_rate=0.1,subsample=0.8,n_estimators=3000,max_depth=6,
                colsample_bytree=1,objective='binary:logistic', n_jobs = -1)

In [ ]:
#fit X_train, y_train
xgbc1.fit(X_train_SMOTE,y_train_SMOTE)

In [ ]:
xgbc1.get_params

In [ ]:
y_train_pred_proba = xgbc1.predict_proba(X_train_SMOTE)[:,1]
# y_valid_pred_proba = xgbc1.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc1.predict_proba(X_test)[:,1]

y_train_pred = xgbc1.predict(X_train_SMOTE)
# y_valid_pred = xgbc1.predict(X_valid)
y_test_pred = xgbc1.predict(X_test)

print('Tuned XGBoost')
print('Training:')
print_report(y_train_SMOTE, y_train_pred, y_train_pred_proba)
# print('Validation:')
# print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)

In [ ]:
confusion_matrix(y_train_SMOTE, y_train_pred)

In [ ]:
confusion_matrix(y_test, y_test_pred)

In [ ]:
sorted(zip(X_train_SMOTE.columns,xgbc1.feature_importances_), key= lambda t:t[1], reverse=True)

In [ ]:
eli5.show_weights(xgbc1)

In [ ]:
# X_test['predicted_probaility'] = y_test_pred_proba
# X_test['actual_readmitted'] = y_test
# X_test['predicted_readmitted'] = y_test_pred

In [ ]:
X_test.sort_values(by='predicted_probaility')

In [ ]:
#adding the quantiles as another columns
# X_test['Quantile_rank'] = pd.qcut(X_test['predicted_probaility'],10,labels=False)

### LightGBM

**Base lightgbm (with SMOTE)**

In [ ]:
# del X_test['predicted_probaility']
# del X_test['actual_readmitted']
# del X_test['predicted_readmitted']
# del X_test['Quantile_rank']

In [ ]:
lgbbase = lgb.LGBMClassifier()
lgbbase.fit(X_train_SMOTE,y_train_SMOTE)

In [ ]:
print("")
#print('='*25,'Results','='*25)
print('')
#print("Score best parameters: ", min(loss)*-1)
#print("Best parameters: ", best_param)
#print("Test Score: ", clf_best.score(X_test, y_test))
#print("Time elapsed: ", time.time() - start)
#print("Parameter combinations evaluated: ", num_eval)
auc = roc_auc_score(y_test, lgbbase.predict_proba(X_test)[:,1])
auct = roc_auc_score(y_train_SMOTE, lgbbase.predict_proba(X_train_SMOTE)[:,1])
print('AUC train is: ',auct)
print('AUC test is: ',auc)
print_report(y_test,lgbbase.predict(X_test),lgbbase.predict_proba(X_test)[:,1])
cmlgbb = pd.DataFrame(confusion_matrix(y_test,lgbbase.predict(X_test)))
cmlgbb

In [ ]:
confusion_matrix(y_train_SMOTE, clf_best.predict(X_train_SMOTE))

**Tune LightGBM SMOTE: Bayesian Optimization** 

In [ ]:
#declare objective function for the optimizer
def objective_function(params):
    clf = lgb.LGBMClassifier(**params)
    skf = ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=99)
    score = cross_val_score(clf, X_train_SMOTE, y_train_SMOTE, scoring = 'roc_auc', cv=skf, n_jobs=1)
    return {'loss': 1-score.mean(), 'status': STATUS_OK}

In [ ]:
#parameter space and minimizer

num_eval = 100
#params to consider
param_hyperopt= {
                #'is_unbalanced': True,
                'objective':'binary',
                'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.09)),
                'max_depth': scope.int(hp.quniform('max_depth', 25, 80, 1)), #5,19,1
                'min_data_in_leaf':scope.int(hp.quniform('min_data_in_leaf', 50, 60, 5)),
                'n_estimators': scope.int(hp.quniform('n_estimators', 100, 200, 20)), #5,55,1  180,220
                'num_leaves': scope.int(hp.quniform('num_leaves', 100, 160, 1)), #5,50,1
                'boosting_type': 'goss', #hp.choice('boosting_type', ['gbdt','dart','goss']), #check lightgbm for types
                'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0),
                'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.5), #ridge
                #'drop_rate': hp.uniform('drop_rate', 0.0, 1.0),
                'reg_alpha': hp.uniform('reg_alpha', 0.10, 0.15),
                #'max_bin': scope.int(hp.quniform('max_bin', 255, 265, 1)),
                #'scale_pos_weight': hp.uniform('scale_pos_weight', 1,1.1),
                #'early_stopping_round': scope.int(hp.quniform('early_stopping_round', 100, 101, 1)),
                'metric': 'auc'
                }

#loss minimizer, then store parameters
trials = Trials()
best_param = fmin(objective_function, 
                  param_hyperopt, 
                  algo=tpe.suggest, 
                  max_evals=num_eval, 
                  trials=trials,
                  rstate= np.random.RandomState(1))
loss = [x['result']['loss'] for x in trials.trials]
best_param_values = [x for x in best_param.values()]

In [ ]:
best_param

In [ ]:
#fit based off the optimizer above

if best_param_values[0] == 0:
    boosting_type = 'gbdt'

elif best_param_values[0] == 1:
    boosting_type = 'dart'

else:
    #best_param_values[0] == 2
    boosting_type = 'goss'    
    
# else:
#     boosting_type = 'goss'
        
    
clf_smote = lgb.LGBMClassifier(
                              #is_unbalanced = True,
                              boosting_type=boosting_type,
                              colsample_bytree = best_param['colsample_bytree'],
                              #drop_rate = best_param['drop_rate'],
                              learning_rate = best_param['learning_rate'],
                              #max_bin = int(best_param['max_bin']),
                              min_data_in_leaf = int(best_param['min_data_in_leaf']),
                              max_depth = int(best_param['max_depth']),
                              n_estimators = int(best_param['n_estimators']),
                              num_leaves = int(best_param['num_leaves']),
                              reg_alpha = best_param['reg_alpha'],
                              reg_lambda = best_param['reg_lambda'],
                              objective = 'binary',
                              metric = 'auc',
                              #scale_pos_weight = 1 #best_param['scale_pos_weight'], #4
#                               bagging_frequency = 1,
#                               pos_bagging_fraction = .99,
#                               neg_bagging_fraction = .3
                              
                              #early_stopping_round = int(best_param['early_stopping_round'])
                              )
                                  
clf_smote.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
## print("")
print('='*25,'Results','='*25)
print('')
print("Score best parameters: ", min(loss)*-1)
print("Best parameters: ", best_param)
print("Test Score: ", clf_smote.score(X_test, y_test))
#print("Time elapsed: ", time.time() - start)
print("Parameter combinations evaluated: ", num_eval)
auc = roc_auc_score(y_test, clf_smote.predict_proba(X_test)[:,1])
auct = roc_auc_score(y_train, clf_smote.predict_proba(X_train)[:,1])
print('AUC train is: ',auct)
print('AUC test is: ',auc)
print_report(y_test,clf_smote.predict(X_test),clf_smote.predict_proba(X_test)[:,1])
cmlgbm = pd.DataFrame(confusion_matrix(y_test,clf_smote.predict(X_test)))
cmlgbm

**Tune LightGBM NO SMOTE: Bayesian Optimization** 

In [ ]:
num_eval = 100
#params to consider
param_hyperopt= {
                'is_unbalanced': True,
                'objective':'binary',
                'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.09)),
                'max_depth': scope.int(hp.quniform('max_depth', 25, 80, 1)), #5,19,1
                'min_data_in_leaf':scope.int(hp.quniform('min_data_in_leaf', 50, 60, 5)),
                'n_estimators': scope.int(hp.quniform('n_estimators', 100, 300, 20)), #5,55,1  180,220
                'num_leaves': scope.int(hp.quniform('num_leaves', 100, 160, 1)), #5,50,1
                'boosting_type': 'goss', #hp.choice('boosting_type', ['gbdt','dart','goss']), #check lightgbm for types
                'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0),
                'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.5), #ridge
                #'drop_rate': hp.uniform('drop_rate', 0.0, 1.0),
                'reg_alpha': hp.uniform('reg_alpha', 0.10, 0.15),
                #'max_bin': scope.int(hp.quniform('max_bin', 255, 265, 1)),
                #'scale_pos_weight': hp.uniform('scale_pos_weight', 1,1.1),
                #'early_stopping_round': scope.int(hp.quniform('early_stopping_round', 100, 101, 1)),
                'metric': 'auc'
                }

#loss minimizer, then store parameters
trials = Trials()
best_param = fmin(objective_function, 
                  param_hyperopt, 
                  algo=tpe.suggest, 
                  max_evals=num_eval, 
                  trials=trials,
                  rstate= np.random.RandomState(1))
loss = [x['result']['loss'] for x in trials.trials]
best_param_values = [x for x in best_param.values()]

In [ ]:
best_param

In [ ]:
if best_param_values[0] == 0:
    boosting_type = 'gbdt'

elif best_param_values[0] == 1:
    boosting_type = 'dart'

else:
    #best_param_values[0] == 2
    boosting_type = 'goss'    
    
# else:
#     boosting_type = 'goss'
        
    
clf_best = lgb.LGBMClassifier(
                              #is_unbalanced = True,
                              boosting_type=boosting_type,
                              colsample_bytree = best_param['colsample_bytree'],
                              #drop_rate = best_param['drop_rate'],
                              learning_rate = best_param['learning_rate'],
                              #max_bin = int(best_param['max_bin']),
                              min_data_in_leaf = int(best_param['min_data_in_leaf']),
                              max_depth = int(best_param['max_depth']),
                              n_estimators = int(best_param['n_estimators']),
                              num_leaves = int(best_param['num_leaves']),
                              reg_alpha = best_param['reg_alpha'],
                              reg_lambda = best_param['reg_lambda'],
                              objective = 'binary',
                              metric = 'auc',
                              scale_pos_weight = 2 #best_param['scale_pos_weight'], #4
#                               bagging_frequency = 1,
#                               pos_bagging_fraction = .99,
#                               neg_bagging_fraction = .3
                              
                              #early_stopping_round = int(best_param['early_stopping_round'])
                              )
                                  
clf_best.fit(X_train, y_train)

In [ ]:
## print("")
print('='*25,'Results','='*25)
print('')
print("Score best parameters: ", min(loss)*-1)
print("Best parameters: ", best_param)
print("Test Score: ", clf_best.score(X_test, y_test))
#print("Time elapsed: ", time.time() - start)
print("Parameter combinations evaluated: ", num_eval)
auc = roc_auc_score(y_test, clf_best.predict_proba(X_test)[:,1])
auct = roc_auc_score(y_train, clf_best.predict_proba(X_train)[:,1])
print('AUC train is: ',auct)
print('AUC test is: ',auc)
print_report(y_test,clf_best.predict(X_test),clf_best.predict_proba(X_test)[:,1])
cmlgbm = pd.DataFrame(confusion_matrix(y_test,clf_best.predict(X_test)))
cmlgbm

**SELECTED (70% whole train)**

In [ ]:
clf_chosen = lgb.LGBMClassifier(
                              is_unbalanced = True,
                              boosting_type='goss',
                              colsample_bytree = 0.5937307464548665,
                              #drop_rate = best_param['drop_rate'],
                              learning_rate = 0.017865274565261144,
                              #max_bin = int(best_param['max_bin']),
                              min_data_in_leaf = 800,
                              max_depth = 61,
                              n_estimators = 3300,
                              num_leaves = 320,
                              reg_alpha = 0.1376473846645497,
                              reg_lambda = 1.3051865999142085,
                              objective = 'binary',
                              metric = 'auc',
                              #scale_pos_weight = best_param['scale_pos_weight'], #4
#                               bagging_frequency = 1,
#                               pos_bagging_fraction = .99,
#                               neg_bagging_fraction = .3
                              
                              #early_stopping_round = int(best_param['early_stopping_round'])
                              )
                                  
clf_chosen.fit(X_train, y_train)

In [ ]:
## print("")
print('='*25,'Results','='*25)
print('')
print("Score best parameters: ", min(loss)*-1)
#print("Best parameters: ", best_param)
print("Test Score: ", clf_chosen.score(X_test, y_test))
#print("Time elapsed: ", time.time() - start)
print("Parameter combinations evaluated: ", num_eval)
auc = roc_auc_score(y_test, clf_chosen.predict_proba(X_test)[:,1])
auct = roc_auc_score(y_train, clf_chosen.predict_proba(X_train)[:,1])
print('AUC train is: ',auct)
print('AUC test is: ',auc)
print_report(y_test,clf_chosen.predict(X_test),clf_chosen.predict_proba(X_test)[:,1])
cmclgbm = confusion_matrix(y_test,clf_chosen.predict(X_test))
cmclgbm

In [ ]:
chosen_weights = eli5.explain_weights(clf_chosen)
chosen_weights

In [ ]:
feat_imp = pd.DataFrame({'weights': [0.174,0.1437,0.1044,0.0756,0.0555,0.0533,0.0532,0.0463,0.025,0.0245,0.0205,0.0177,0.0148,0.0139,0.0128,0.0125,0.012,0.0111,0.011],\
 'features':['Inpatient Visits','Num. lab procedure','Num. .medications','Time in hospital','age','discharged to home','number of diagnoses','Num. procedures','Num. emergency','Num. medicine','Male','discharged to hospital','admitted from urgentcare','No insulin given','Num. outpatient','change_No','med. dosage change','admitted from referral','admission type urgent']})

In [ ]:
#feature_names = feat_imp['features']
#feature_imports = feat_imp['weights']
#most_imp_features = pd.DataFrame([f for f in zip(feature_names,feature_imports)], columns=["Feature", "Importance"]).nlargest(10, "Importance")
feat_imp.sort_values(by="weights", inplace=True, ascending = True)
plt.figure(figsize=(10,6))
plt.barh(range(len(feat_imp)), feat_imp['weights'], align='center', alpha=0.8)
plt.yticks(range(len(feat_imp)), feat_imp['features'], fontsize=14)
plt.xlabel('Importance')
plt.title('Most important features by weight - LightGBM')
plt.show()

In [ ]:
def make_lift(X_test,y_test,best_model):
    
    liftdf = X_test.copy()

    liftdf['predicted_probaility'] = best_model.predict_proba(X_test)[:,1]
    liftdf['predicted_readmitted'] = best_model.predict(X_test)
    liftdf['actual_readmitted'] = y_test
    #adding the quantiles as another columns
    liftdf['Quantile_rank'] = pd.qcut(liftdf['predicted_probaility'],10,labels=np.arange(1,11))
    
    group = liftdf.groupby('Quantile_rank', as_index = False)['actual_readmitted'].agg(['count','sum'])
    group.columns = ['unique patients', 'actual readmitted']
    group.reset_index(inplace = True)
    
    lift_chart = liftdf.groupby('Quantile_rank')['predicted_probaility'].agg(['min','max','mean'])
    lift_chart.columns = ['min_prob','max_prob','avg_prob']
    lift_chart.reset_index(inplace = True)
    
    lift_table = pd.merge(group,lift_chart, how = 'left', on = 'Quantile_rank')
    lift_table['pct_readmitted'] = lift_table['actual readmitted']/lift_table['unique patients']
    
    return lift_table

In [ ]:
lift = make_lift(X_test,y_test,clf_chosen)
lift

In [ ]:
lift.to_csv('dataset_diabetes/liftcurrent.csv',index = False)

In [ ]:
#plot for tuned model

y_score = clf_best.predict_proba(X_test)
fpr_tuned, tpr_tuned, _ = roc_curve(y_test,  y_score[:,1])

# # Plot ROC curve
# plt.plot(fpr_base, tpr_base, label='Base ROC curve (area = %0.3f)' % auc_score_base)
plt.plot(fpr_tuned, tpr_tuned, label='Tuned ROC curve (area = %0.3f)')# % auc_score_tuned)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('ROC AUC Curve')
plt.legend(loc="lower right")
plt.show()